## 2. Viết chương trình tạo menu cho phép người dùng chọn các phương pháp biến đổi ảnh như sau :
### Fast Fouriel
### Butterworth Lowpass Filter
### Butterworth Highpass Filter 
## Khi người dùng ấn phím F, L, H chương trình sẽ thực hiên hàm tương ứng 

In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image

def fast_fourier_transform(image):
    f = np.fft.fft2(image)
    fshift = np.fft.fftshift(f)
    magnitude = 20 * np.log(np.abs(fshift) + 1)
    return magnitude

def butterworth_lowpass(image, d0=30, n=2):
    rows, cols = image.shape
    crow, ccol = rows // 2, cols // 2

    dft = np.fft.fft2(image)
    dft_shift = np.fft.fftshift(dft)

    u = np.arange(rows)
    v = np.arange(cols)
    U, V = np.meshgrid(v, u)
    D = np.sqrt((U - ccol)**2 + (V - crow)**2)

    H = 1 / (1 + (D / d0)**(2*n))
    G = dft_shift * H

    img_back = np.fft.ifft2(np.fft.ifftshift(G))
    img_back = np.abs(img_back)
    return img_back

def butterworth_highpass(image, d0=30, n=2):
    rows, cols = image.shape
    crow, ccol = rows // 2, cols // 2

    dft = np.fft.fft2(image)
    dft_shift = np.fft.fftshift(dft)

    u = np.arange(rows)
    v = np.arange(cols)
    U, V = np.meshgrid(v, u)
    D = np.sqrt((U - ccol)**2 + (V - crow)**2)

    H = 1 - 1 / (1 + (D / d0)**(2*n))
    G = dft_shift * H

    img_back = np.fft.ifft2(np.fft.ifftshift(G))
    img_back = np.abs(img_back)
    return img_back


mapping = {
    'F': ('Fast_Fourier_Transform', fast_fourier_transform),
    'L': ('Butterworth_Lowpass_Filter', butterworth_lowpass),
    'H': ('Butterworth_Highpass_Filter', butterworth_highpass),
}


key = input("Chọn phương pháp (F: Fast Fourier, L: Lowpass, H: Highpass): ").upper()
if key not in mapping:
    print("Phím không hợp lệ.")
    exit()

method_name, func = mapping[key]


input_folder = 'exercise'
output_folder = 'output_menu'
os.makedirs(output_folder, exist_ok=True)


for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        result = func(img)
        result = cv2.normalize(result, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

        output_img_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_{key}.png")
        cv2.imwrite(output_img_path, result)

  
        img_pil = Image.fromarray(img)
        result_pil = Image.fromarray(result)

        print(f"Đang hiển thị ảnh: {filename}")
        img_pil.show(title="Original")
        result_pil.show(title=method_name)

        plt.figure(figsize=(8, 4))
        plt.subplot(1, 2, 1)
        plt.imshow(img, cmap='gray')
        plt.title("Original")
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(result, cmap='gray')
        plt.title(method_name)
        plt.axis('off')

        plt.tight_layout()
        plt.show()


## Bài 3 : 

In [ ]:
import cv2
import numpy as np
import os
import random
import matplotlib.pyplot as plt
from PIL import Image

def inverse(img):
    return 255 - img
def gamma_correction(img, gamma=2.2):
    return np.power(img / 255.0, gamma) * 255
def log_transform(img):
    return np.log1p(img) / np.log(256) * 255
def hist_equalization(img):
    return cv2.equalizeHist(img)
def contrast_stretch(img):
    return ((img - img.min()) / (img.max() - img.min()) * 255).astype(np.uint8)

transform_funcs = [
    ('I', 'Inverse', inverse),
    ('G', 'Gamma Correction', gamma_correction),
    ('L', 'Log Transform', log_transform),
    ('H', 'Histogram Equalization', hist_equalization),
    ('C', 'Contrast Stretching', contrast_stretch)
]


input_folder = 'exercise'
output_folder = 'output_bai3'
os.makedirs(output_folder, exist_ok=True)


for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path)  # Ảnh màu BGR

        
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        channels = [0, 1, 2]
        random.shuffle(channels)
        shuffled_rgb = img_rgb[:, :, channels]

        
        gray = cv2.cvtColor(shuffled_rgb, cv2.COLOR_RGB2GRAY)

        
        key, method_name, func = random.choice(transform_funcs)
        result = func(gray).astype(np.uint8)

      
        output_img_path = os.path.join(output_folder, f"{filename[:-4]}_{key}.png")
        cv2.imwrite(output_img_path, result)

        
        print(f"Ảnh: {filename} | Hoán màu RGB → {channels} | Biến đổi: {method_name}")
        Image.fromarray(gray).show(title="After RGB Shuffle (Grayscale)")
        Image.fromarray(result).show(title=f"{method_name}")

        plt.figure(figsize=(8, 4))
        plt.subplot(1, 2, 1)
        plt.imshow(gray, cmap='gray')
        plt.title("RGB Shuffled → Grayscale")
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(result, cmap='gray')
        plt.title(f"{method_name}")
        plt.axis('off')

        plt.tight_layout()
        plt.show()
